In [5]:
from ultralytics import YOLO
import os
import shutil
import time

# Preparación del Dataset.

### Material a utilizar:

1. Cargar el modelo de deteccion de objetos.
    El modelo lo sacamos del paper de Dillon Reis, "Real-Time Flying Object Detection with YOLOv8"
2. El Dataset lo tenemos descargado en Local.
    Proviene de Roboflow -> [Flying Object Detection](https://universe.roboflow.com/new-workspace-0k81p/flying_object_dataset/dataset/2)

### Pasos a relizar en este notebook:

1. (Paso 1) Las imagenes del dataset vienen con nombres en un formato no valido, asi que le cambiaremos el nombre a todas las imagenes.
2. (Paso 2) Pasamos las imagenes por el detector de objetos y guardamos la imagen detectada segun su categoria.
    - Si detecta un objeto clase "Drone" -> "Drone"
    - Si etecta un objeto de cualquier otra clase -> "No Drone"

In [15]:
model = YOLO('modelos/00_pred_ue.pt')

### Paso 0: Pruebas iniciales para ver con que formato se obtienen los resultados.

In [22]:
img_file =  "./dataset_graficas/images/0000001.jpg"
results = model(source= img_file, save_crop=True, project= './shared_folder', name= 'hoal')
print(results[0].boxes)
if results:
    for r in results:
        print(r.boxes.cls.tolist())
        if r.boxes.cls.tolist():
            print(int(r.boxes.cls.tolist()[0]))
else:
    print("oo")


image 1/1 c:\Users\pabpg\Desktop\MasterBD\TFM\Train_resize\dataset_graficas\images\0000001.jpg: 640x640 (no detections), 1157.1ms
Speed: 7.0ms preprocess, 1157.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to shared_folder\hoal11
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([])
conf: tensor([])
data: tensor([], size=(0, 6))
id: None
is_track: False
orig_shape: (416, 416)
shape: torch.Size([0, 6])
xywh: tensor([], size=(0, 4))
xywhn: tensor([], size=(0, 4))
xyxy: tensor([], size=(0, 4))
xyxyn: tensor([], size=(0, 4))
[]


In [16]:
results = model(source= "./imagenes_v2/0010624.jpg")

for result in results:
    boxes = result.boxes  
    masks = result.masks  
    keypoints = result.keypoints
    probs = result.boxes.conf
    obb = result.obb 
    print(boxes)
    result.show()


image 1/1 c:\Users\pabpg\Desktop\MasterBD\TFM\Train_resize\imagenes_v2\0010624.jpg: 384x640 2 drones, 670.8ms
Speed: 3.1ms preprocess, 670.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to dataset_recortado\test\temp\0001538
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([38., 38.])
conf: tensor([0.6945, 0.6395])
data: tensor([[8.1370e+02, 1.6566e+02, 9.1926e+02, 2.3031e+02, 6.9446e-01, 3.8000e+01],
        [1.7470e+02, 1.8672e+02, 2.6978e+02, 2.4052e+02, 6.3952e-01, 3.8000e+01]])
id: None
is_track: False
orig_shape: (720, 1280)
shape: torch.Size([2, 6])
xywh: tensor([[866.4811, 197.9829, 105.5547,  64.6478],
        [222.2412, 213.6226,  95.0732,  53.8029]])
xywhn: tensor([[0.6769, 0.2750, 0.0825, 0.0898],
        [0.1736, 0.2967, 0.0743, 0.0747]])
xyxy: tensor([[813.7037, 165.6590, 919.2584, 230.3068],
        [174.7046, 186.7211, 269.7778, 240.5241]])
xyxyn: tensor([[0.6357, 0.2301, 0.7182, 0.3199],
        [0.1365, 0.2593,

### Paso 1: Renombrar las imagenes del dataset para que no tengan un nombre tan largo.

In [2]:

directory = './dataset_original/test/labels'


files = [f for f in os.listdir(directory) if f.endswith('.txt')]
files.sort()


for idx, filename in enumerate(files, start=1):
    new_name = f"{idx:07}.txt"
    
    src = os.path.join(directory, filename)
    dst = os.path.join(directory, new_name)
    
    os.rename(src, dst)

print("Renombrado completo")


Renombrado completo


### Paso 2: Recortar las imagenes detectadas y guardarlas segun su clase

In [31]:


img_dir = './dataset/valid/images'

# Carpeta donde se guardarán inicialmente los crops
initial_save_path = './dataset_recortado/valid/temp'

j = 200001


for img_file in os.listdir(img_dir):
    
    if img_file.endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(img_dir, img_file)

        results = model.predict(source=img_path, save_crop=True, project=initial_save_path, name=os.path.splitext(img_file)[0])

        # Obtener las clases detectadas
        for result in results:
            crops_dir = os.path.join(initial_save_path, os.path.splitext(img_file)[0], 'crops')

            for i, box in enumerate(result.boxes):
                class_id = int(box.cls[0])
                class_name = model.names[class_id]

                # Definir la carpeta destino basada en la clase detectada
                if class_name == 'drone':
                    destination_folder = './dataset_recortado/valid/drone'
                    print(f'drone {j}')
                else:
                    destination_folder = './dataset_recortado/valid/no_drone'

                # Crear la carpeta destino si no existe
                os.makedirs(destination_folder, exist_ok=True)

                # Ruta del crop actual
                crop_file_name = img_file
                crop_file_name_final = f'img{j}.jpg'
                crop_path = os.path.join(crops_dir, class_name, crop_file_name)
                j += 1

                if os.path.exists(crop_path):
                    shutil.move(crop_path, os.path.join(destination_folder, crop_file_name_final))
                    time.sleep(0.1)





image 1/1 c:\Users\pabpg\Desktop\MasterBD\TFM\Train_resize\dataset\valid\images\0000001.jpg: 640x640 1 c-helicopter, 1080.6ms
Speed: 6.0ms preprocess, 1080.6ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to dataset_recortado\valid\temp\0000001

image 1/1 c:\Users\pabpg\Desktop\MasterBD\TFM\Train_resize\dataset\valid\images\0000002.jpg: 640x640 1 p-airplane, 1059.7ms
Speed: 5.0ms preprocess, 1059.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to dataset_recortado\valid\temp\0000002

image 1/1 c:\Users\pabpg\Desktop\MasterBD\TFM\Train_resize\dataset\valid\images\0000003.jpg: 640x640 1 c-helicopter, 1112.9ms
Speed: 5.1ms preprocess, 1112.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to dataset_recortado\valid\temp\0000003

image 1/1 c:\Users\pabpg\Desktop\MasterBD\TFM\Train_resize\dataset\valid\images\0000004.jpg: 640x640 1 c-helicopter, 1129.8ms
Speed: 5.0ms preprocess, 1129.8ms i

In [28]:

img_dir = './imagenes'
initial_save_path = './dataset_recortado/test/temp'

j = 300001


for img_file in os.listdir(img_dir):
    
    if img_file.endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(img_dir, img_file)
        results = model.predict(source=img_path, save_crop=True, project=initial_save_path, name=os.path.splitext(img_file)[0])

        for result in results:
            crops_dir = os.path.join(initial_save_path, os.path.splitext(img_file)[0], 'crops')

            # Iterar sobre cada resultado y mover los crops a las carpetas correspondientes
            for i, box in enumerate(result.boxes):

                # Obtener la clase del objeto detectado
                class_id = int(box.cls[0])
                class_name = model.names[class_id]

                # Definir la carpeta destino basada en la clase detectada
                if class_name == 'drone':
                    destination_folder = './dataset_recortado/test/drone'
                    print(f'drone {j}')
                else:
                    destination_folder = './dataset_recortado/test/no_drone'

                # Crear la carpeta destino si no existe
                os.makedirs(destination_folder, exist_ok=True)

                # Ruta del crop actual
                crop_file_name = img_file
                crop_file_name_final = f'img{j}.jpg'
                crop_path = os.path.join(crops_dir, class_name, crop_file_name)
                j += 1

                if os.path.exists(crop_path):
                    shutil.move(crop_path, os.path.join(destination_folder, crop_file_name_final))
                    time.sleep(0.1)



image 1/1 c:\Users\pabpg\Desktop\MasterBD\TFM\Train_resize\imagenes\Autumn_Drone.jpg: 448x640 1 drone, 868.1ms
Speed: 5.1ms preprocess, 868.1ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to dataset_recortado\temp\be771daa41.jpg

image 1/1 c:\Users\pabpg\Desktop\MasterBD\TFM\Train_resize\imagenes\avion.jpg: 320x640 1 p-airplane, 617.4ms
Speed: 2.0ms preprocess, 617.4ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 640)
Results saved to dataset_recortado\temp\9424fa904b.jpg

image 1/1 c:\Users\pabpg\Desktop\MasterBD\TFM\Train_resize\imagenes\avion2.jpg: 640x640 1 Be200, 1212.6ms
Speed: 3.5ms preprocess, 1212.6ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to dataset_recortado\temp\06401e8400.jpg

image 1/1 c:\Users\pabpg\Desktop\MasterBD\TFM\Train_resize\imagenes\avion3.jpg: 640x640 1 B52, 1150.0ms
Speed: 5.6ms preprocess, 1150.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results